# Fetch Historic Energy Demand for Benchmarking & Training

In [ ]:
import sys
!{sys.executable} -m pip install git+https://github.com/tomharvey/ElexonDataPortal.git@pandas-version-tz-compare

In [1]:
from ElexonDataPortal import api
import pandas as pd
from secret import api_key
from datetime import date, timedelta

In [2]:
client = api.Client(api_key)

In [3]:
# Data Available
pd.Series(client.methods)

get_B0610                                    Actual Total Load per Bidding Zone
get_B0620                        Day-Ahead Total Load Forecast per Bidding Zone
get_B0630                       Week-Ahead Total Load Forecast per Bidding Zone
get_B0640                      Month-Ahead Total Load Forecast Per Bidding Zone
get_B0650                       Year Ahead Total Load Forecast per Bidding Zone
get_B0710                           Planned Unavailability of Consumption Units
get_B0720                     Changes In Actual Availability Of Consumption ...
get_B0810                                            Year Ahead Forecast Margin
get_B0910                                    Expansion and Dismantling Projects
get_B1010                       Planned Unavailability In The Transmission Grid
get_B1020                     Changes In Actual Availability In The Transmis...
get_B1030                     Changes In Actual Availability of Offshore Gri...
get_B1320                         Conges

In [ ]:
# Fetch Benchmarking Dataset
dfs = []
current_date = date(2023, 11, 30)
target_date = date(2020, 11, 30)
while current_date > target_date:
    new_date = current_date - timedelta(weeks=4)
    dfs.append(client.get_SYSDEM(new_date, current_date))
    current_date = new_date

In [ ]:
df_full = pd.concat(dfs)
df_full.index = pd.to_datetime(df_full['local_datetime'])
df_full = df_full.loc[df_full['recordType'] == 'ITSDO']
s_full = df_full['demand'].astype('float64')
new_date_index = pd.date_range(start='2020-12-01', end='2023-11-30', freq='30T', tz='UTC')
s_full_wo_duplicate = s_full.loc[~s_full.index.duplicated(keep='first')]
s_reindex = s_full_wo_duplicate.reindex(new_date_index)
s_reindex_interp = s_reindex.interpolate(method='linear')

In [ ]:
s_reindex_interp.to_pickle('../00_Data/historic_demand.pkl')